In [22]:
import json
import pandas as pd
import sys

def to_df(filename):
    with open(filename, 'r') as json_file:
        data = json.load(json_file)

    df = pd.DataFrame(data)
    df.to_csv(f'{filename.split(".json")[0]}.csv', index=False, sep=';')
    return df
scan = to_df('crypto_rank.json')
with open("contract_data.json", 'r') as json_file:
        data = json.load(json_file)

In [35]:
codes = dict()
for key in data:
    try:
        print(len(data[key][0]['result']))
        codes[key] = data[key][0]['result']
    except IndexError:
        print(data[key])
codes
pd.DataFrame(codes)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
[]
1
1
[]
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
[]
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
[]
[]
1
1
1
[]
1
1
1
[]
[]
1
1
[]
1
1
1
[]
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
[]
[]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
1
1
1
1
[]
[]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
[]
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
[]
1
1
1
[]
1
1
1
1
1
1
[]
1
1
1
[]
1
1
1
1
1
[]
[]
1
[]
[]
1
1
[]
1
1
1
1
1
1
1
1
[]
[]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
[]
1
[]
1
1
1
1
1
[]
1
1
1
1
[]
1
1
[]
1
1
1
1
1
1
1

,dao-token,my-neighbor-alice,sweat-economy,gamefi,victoria-vr,xcad-network,opulous,lto-network,vaiot,orionprotocol,...,fintropy,oren-game,merkle-network,morphie-network,furukuru,polylastic,mist-nft,orakuru,algopainter,nft-alley
0,{'SourceCode': '// File: openzeppelin-solidity...,{'SourceCode': '/** *Submitted for verificat...,{'SourceCode': '// File: @openzeppelin/contrac...,"{'SourceCode': '{{ ""language"": ""Solidity"", ...","{'SourceCode': '{""Address.sol"":{""content"":""// ...",{'SourceCode': '// File: @openzeppelin/contrac...,"{'SourceCode': '{{ ""language"": ""Solidity"", ...","{'SourceCode': '{{ ""language"": ""Solidity"", ...",{'SourceCode': '// Sources flattened with hard...,"{'SourceCode': '{""Context.sol"":{""content"":""pra...",...,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",{'SourceCode': '/** *Submitted for verificat...,{'SourceCode': 'pragma solidity ^0.6.0; // SP...,{'SourceCode': 'pragma solidity 0.5.16; c...,{'SourceCode': '// File: @openzeppelin/contrac...,"{'SourceCode': '{{ ""language"": ""Solidity"", ...","{'SourceCode': '{{ ""language"": ""Solidity"", ...","{'SourceCode': '{{ ""language"": ""Solidity"", ...",{'SourceCode': '// File: @openzeppelin/contrac...,{'SourceCode': '// File: openzeppelin-solidity...


In [128]:
n_data = list()
n_data = []
for key in data:
    if data[key] == []:
        continue
    n_data.append(data[key][0]['result'][0])
    n_data[-1]['name']

with open("crypto_rank.json", 'r') as json_file:
        cap_data = json.load(json_file)

def add_mark(item, mark):
    for platform in cap_data['items']:
        if item['name'] in platform['props']['pageProps']['fallbackCoins']:
            try:
                item[mark] = platform['props']['pageProps']['fallbackCoins'][item['name']][mark]
            except:
                item[mark] = None
for item in n_data:
    add_mark(item, 'histPrices')
    add_mark(item, 'priceUsd')
    add_mark(item, 'volume24h')

In [129]:
k = pd.DataFrame(n_data)
k

,SourceCode,ABI,ContractName,CompilerVersion,OptimizationUsed,Runs,ConstructorArguments,EVMVersion,Library,LicenseType,Proxy,Implementation,SwarmSource,name,histPrices,priceUsd,volume24hUsd,volume24h
0,// File: openzeppelin-solidity/contracts/token...,"[{""constant"":true,""inputs"":[],""name"":""name"",""o...",Token,v0.5.0+commit.1d4f565a,1,200,,Default,,None,0,,bzzr://acb7aeee9acc4a9dd1d8720180c22d141d340cb...,dao-token,"{'YTD': {'USD': 0.83514395776, 'BTC': 5.050591...",0.661125,1.325299e+06,2.004610e+06
1,/**\r\n *Submitted for verification at Ethersc...,"[{""inputs"":[{""internalType"":""address"",""name"":""...",AliceToken,v0.6.12+commit.27d51765,1,200,000000000000000000000000894a1a70311cd19a3ef33a...,Default,,None,0,,ipfs://9a1945593950cd4e682baee35dae92b729b1864...,my-neighbor-alice,"{'7D': {'USD': 0.902162515837, 'BTC': 2.596854...",0.966153,1.085961e+07,1.124005e+07
2,// File: @openzeppelin/contracts/proxy/beacon/...,"[{""inputs"":[{""internalType"":""address"",""name"":""...",BridgeToken,v0.8.4+commit.c7e474f2,1,200,0000000000000000000000003ee18b2214aff97000d974...,Default,,Apache-2.0,1,0x3ee18b2214aff97000d974cf647e7c347e8fa585,ipfs://3e23ac4f79a95247999f8e0df94ea5bd5e38561...,sweat-economy,"{'30D': {'USD': 0.010181411585, 'BTC': 3.63972...",0.009076,1.100539e+06,1.212603e+08
3,"{{\r\n ""language"": ""Solidity"",\r\n ""sources""...","[{""inputs"":[{""internalType"":""address"",""name"":""...",GameFi,v0.8.0+commit.c7dfd78e,1,200,000000000000000000000000cd25269326982f51064292...,Default,,,0,,,gamefi,"{'30D': {'USD': 1.529610709174, 'BTC': 5.46816...",3.813445,2.079105e+05,5.452039e+04
4,"{""Address.sol"":{""content"":""// SPDX-License-Ide...","[{""inputs"":[{""internalType"":""address"",""name"":""...",VictoriaVR,v0.6.8+commit.0bbfe453,0,200,00000000000000000000000010ee4afe37a863a124d0c9...,Default,,MIT,0,,ipfs://2794c0c2e36e7ec76c62fe4f3893f44701bfe2c...,victoria-vr,"{'7D': {'USD': 0.006638558879, 'BTC': 1.910893...",0.007203,3.825044e+05,5.310346e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,"{{\r\n ""language"": ""Solidity"",\r\n ""sources""...","[{""inputs"":[{""internalType"":""string"",""name"":""_...",PolylasticV3,v0.8.4+commit.c7e474f2,1,200,0000000000000000000000000000000000000000000000...,Default,,,0,,,polylastic,"{'6M': {'USD': 7.643957e-05, 'BTC': 2.7531214e...",0.000022,NaN,1.411875e+10
984,"{{\r\n ""language"": ""Solidity"",\r\n ""sources""...","[{""constant"":true,""inputs"":[],""name"":""name"",""o...",Mist,v0.5.11+commit.c082d0b4,1,200,,petersburg,,,0,,,mist-nft,"{'7D': {'USD': 0.001543412485, 'BTC': 4.454585...",0.001263,NaN,3.956386e+07
985,"{{\r\n ""language"": ""Solidity"",\r\n ""sources""...","[{""inputs"":[{""internalType"":""address"",""name"":""...",Orakuru,v0.6.12+commit.27d51765,1,200,0000000000000000000000007f8ed9e26d194cbfc4ceb4...,Default,,,0,,,orakuru,{},NaN,NaN,0.000000e+00
986,// File: @openzeppelin/contracts/utils/Context...,"[{""inputs"":[{""internalType"":""string"",""name"":""n...",AlgoPainterToken,v0.7.4+commit.3f05b770,0,200,0000000000000000000000000000000000000000000000...,Default,,MIT,0,,ipfs://90c03c897c6bb64869694e5683fa84e425b0ece...,algopainter,{},NaN,NaN,0.000000e+00


In [103]:
k

,0,1,2,3,4
0,{'SourceCode': '// File: openzeppelin-solidity...,dao-token,dao-token,dao-token,dao-token
1,{'SourceCode': '/** *Submitted for verificat...,my-neighbor-alice,my-neighbor-alice,my-neighbor-alice,my-neighbor-alice
2,{'SourceCode': '// File: @openzeppelin/contrac...,sweat-economy,sweat-economy,sweat-economy,sweat-economy
3,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",gamefi,gamefi,gamefi,gamefi
4,"{'SourceCode': '{""Address.sol"":{""content"":""// ...",victoria-vr,victoria-vr,victoria-vr,victoria-vr
...,...,...,...,...,...
983,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",polylastic,polylastic,polylastic,polylastic
984,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",mist-nft,mist-nft,mist-nft,mist-nft
985,"{'SourceCode': '{{ ""language"": ""Solidity"", ...",orakuru,orakuru,orakuru,orakuru
986,{'SourceCode': '// File: @openzeppelin/contrac...,algopainter,algopainter,algopainter,algopainter
